# Create Annual Calibrated Composite Images for Each RTS Polygon

## Set Up Environment

In [ ]:
import ee
ee.Initialize()

In [ ]:
# Import Libraries
import geemap
import os
from pprint import pprint
import math
import statistics
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely as shp
import xarray as xr
import rioxarray as rxr
from datetime import datetime
from collections import Counter
import re
from google.cloud import storage

In [ ]:
# Set up access to abrupt_thaw
storage_client = storage.Client(project="AbruptThawMapping")
abrupt_thaw = storage_client.get_bucket('abrupt_thaw')

## Define Functions

In [ ]:
# Set Properties to allow filtering
def setID(image):
    img_id = image.id();
    img_prop = image.setMulti({'ID': img_id});
    img_prop = ee.Image(img_prop);
    return img_prop;

In [ ]:
# function to get UTM zone from WGS84 lat and lon
def utm_from_wgs84(lon, lat):
    #Special Cases for Norway and Svalbard
    if (lat > 55 and lat < 64 and lon > 2 and lon < 6):
        return 32
    elif (lat > 71 and lon >= 6 and lon < 9):
        return 31
    elif (lat > 71 and ((lon >= 9 and lon < 12) or (lon >= 18 and lon < 21))):
        return 33
    elif (lat > 71 and ((lon >= 21 and lon < 24) or (lon >= 30 and lon < 33))):
        return 35
    # Rest of the world
    elif (lon >= -180 and lon <= 180):
        return 32600 + (math.floor((lon + 180) / 6) % 60) + 1 # 32600 for northern hemisphere
    else:
        raise ValueError('Cannot figure out UTM zone from given Lat: {0}, Lon: {1}.'.format(lat, lon))

## Import Data and Prepare Visualization Parameters

In [ ]:
# Import Planet Data GCS
planet = ee.ImageCollection('projects/abruptthawmapping/assets/yg_val_regions_imagery_calibrated')
planet = planet.map(setID)

In [ ]:
planet.first().getInfo()

In [ ]:
# Prep Map
Map = geemap.Map()
Map.centerObject(planet)

In [ ]:
# View the imagery
vis_params_imagery = {
    'min': [470, 415, 280],'max': [1180, 930, 750],
    'bands': ['red', 'green', 'blue'],
    'gamma': 0.9
}

## Prepare Data

In [ ]:
# Mask the data to values greater than 0 (which is how nodata gets imported into GEE by default)
def mask_0(image):
    mask = image.gt(0)
    return image.updateMask(mask)

planet = planet.map(mask_0)

In [ ]:
years = [2017, 2018, 2019, 2020, 2021]

## Create Annual Composites

In [ ]:
# create a composite image across all regions for each year individually and all years combined
planet_composite_2017 = planet.filter(ee.Filter.stringContains('ID', '2017')).median()
planet_composite_2018 = planet.filter(ee.Filter.stringContains('ID', '2018')).median()
planet_composite_2019 = planet.filter(ee.Filter.stringContains('ID', '2019')).median()
planet_composite_2020 = planet.filter(ee.Filter.stringContains('ID', '2020')).median()
planet_composite_2021 = planet.filter(ee.Filter.stringContains('ID', '2021')).median()

planet_composite_all = planet.median()

## Map Composites

In [ ]:
# Add composites to the map as one layer
Map.addLayer(planet_composite_2017,
             vis_params_imagery,
             '2017 Composites')
Map.addLayer(planet_composite_2018,
             vis_params_imagery,
             '2018 Composites')
Map.addLayer(planet_composite_2019,
             vis_params_imagery,
             '2019 Composites')
Map.addLayer(planet_composite_2020,
             vis_params_imagery,
             '2020 Composites')
Map.addLayer(planet_composite_2021,
             vis_params_imagery,
             '2020 Composites')
Map.addLayer(planet_composite_all,
             vis_params_imagery,
             'All')

In [ ]:
Map

## Export Annual Composites

In [ ]:
# Import shapefile with AOI (multipolygon)
aoi = gpd.read_file("/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/bboxes/yg_validation_bboxes.shp")
aoi['region'] = [0, 1, 2, 3]
# convert to json for planet data search
sites = json.loads(aoi.to_json()) # if multiple sites

In [ ]:
zones = pd.DataFrame(columns = ['region', 'utm_zone'])
for idx, region in enumerate(aoi.geometry):
    region_zones = []
    for x, y in zip(region.exterior.coords.xy[0], region.exterior.coords.xy[1]):
        region_zones.append(utm_from_wgs84(x, y))
        
    region_zones = round(statistics.median(region_zones))
    temp_df = pd.DataFrame({'region': [idx],
                            'utm_zone': [region_zones]})
    zones = pd.concat([zones, temp_df])
zones = zones.set_index('region')  
zones

In [ ]:
# # Export Composites to Drive (2017)
# for region in aoi.region:
#     name = 'yg_val_regions_composite_region_' + str(region) + '_2017'
#     geometry = sites['features'][region]['geometry']['coordinates']
#     scale = 3
#     crs = 'EPSG:' + str(zones.iloc[region].utm_zone)
#     task = ee.batch.Export.image.toCloudStorage(
#         image = planet_composite_2017,
#         description = name,
#         bucket = 'abrupt_thaw',
#         fileNamePrefix = 'planet_processing/data/yg_val_regions/calibrated_composites/' + name,
#         crs = crs,
#         region = geometry,
#         scale = scale,
#         maxPixels = 1e13,
#         fileFormat = 'GeoTIFF',
#         formatOptions = {'cloudOptimized': True}
#     )
#     task.start()

In [ ]:
# # Export Composites to Drive (2018)
# for region in aoi.region:
#     name = 'yg_val_regions_composite_region_' + str(region) + '_2018'
#     geometry = sites['features'][region]['geometry']['coordinates']
#     scale = 3
#     crs = 'EPSG:' + str(zones.iloc[region].utm_zone)
#     task = ee.batch.Export.image.toCloudStorage(
#         image = planet_composite_2018,
#         description = name,
#         bucket = 'abrupt_thaw',
#         fileNamePrefix = 'planet_processing/data/yg_val_regions/calibrated_composites/' + name,
#         crs = crs,
#         region = geometry,
#         scale = scale,
#         maxPixels = 1e13,
#         fileFormat = 'GeoTIFF',
#         formatOptions = {'cloudOptimized': True}
#     )
#     task.start()

In [ ]:
# # Export Composites to Drive (2019)
# for region in aoi.region:
#     name = 'yg_val_regions_composite_region_' + str(region) + '_2019'
#     geometry = sites['features'][region]['geometry']['coordinates']
#     scale = 3
#     crs = 'EPSG:' + str(zones.iloc[region].utm_zone)
#     task = ee.batch.Export.image.toCloudStorage(
#         image = planet_composite_2019,
#         description = name,
#         bucket = 'abrupt_thaw',
#         fileNamePrefix = 'planet_processing/data/yg_val_regions/calibrated_composites/' + name,
#         crs = crs,
#         region = geometry,
#         scale = scale,
#         maxPixels = 1e13,
#         fileFormat = 'GeoTIFF',
#         formatOptions = {'cloudOptimized': True}
#     )
#     task.start()

In [ ]:
# # Export Composites to Drive (2020)
# for region in aoi.region:
#     name = 'yg_val_regions_composite_region_' + str(region) + '_2020'
#     geometry = sites['features'][region]['geometry']['coordinates']
#     scale = 3
#     crs = 'EPSG:' + str(zones.iloc[region].utm_zone)
#     task = ee.batch.Export.image.toCloudStorage(
#         image = planet_composite_2020,
#         description = name,
#         bucket = 'abrupt_thaw',
#         fileNamePrefix = 'planet_processing/data/yg_val_regions/calibrated_composites/' + name,
#         crs = crs,
#         region = geometry,
#         scale = scale,
#         maxPixels = 1e13,
#         fileFormat = 'GeoTIFF',
#         formatOptions = {'cloudOptimized': True}
#     )
#     task.start()

In [ ]:
# # Export Composites to Drive (2021)
# for region in aoi.region:
#     name = 'yg_val_regions_composite_region_' + str(region) + '_2021'
#     geometry = sites['features'][region]['geometry']['coordinates']
#     scale = 3
#     crs = 'EPSG:' + str(zones.iloc[region].utm_zone)
#     task = ee.batch.Export.image.toCloudStorage(
#         image = planet_composite_2021,
#         description = name,
#         bucket = 'abrupt_thaw',
#         fileNamePrefix = 'planet_processing/data/yg_val_regions/calibrated_composites/' + name,
#         crs = crs,
#         region = geometry,
#         scale = scale,
#         maxPixels = 1e13,
#         fileFormat = 'GeoTIFF',
#         formatOptions = {'cloudOptimized': True}
#     )
#     task.start()

In [ ]:
# # Export Composites to Drive (all)
# for region in aoi.region:
#     name = 'yg_val_regions_composite_region_' + str(region) + '_all'
#     geometry = sites['features'][region]['geometry']['coordinates']
#     scale = 3
#     crs = 'EPSG:' + str(zones.iloc[region].utm_zone)
#     task = ee.batch.Export.image.toCloudStorage(
#         image = planet_composite_all,
#         description = name,
#         bucket = 'abrupt_thaw',
#         fileNamePrefix = 'planet_processing/data/yg_val_regions/calibrated_composites/' + name,
#         crs = crs,
#         region = geometry,
#         scale = scale,
#         maxPixels = 1e13,
#         fileFormat = 'GeoTIFF',
#         formatOptions = {'cloudOptimized': True}
#     )
#     task.start()